In [2]:
import pandas as pd
import numpy as np
import tabula
import math
import os
from datetime import date

In [81]:
class VacsTable:
    """
    VacsTable class defines a vacants table and methods to clean that table
    """
    table_types = {'vacants': [        #list of vacants tables admitted. 
                            'LD-PA common',   #ld-pa vacants usually generate (2, 14) DFs (need to separate?)
                            'CM common',      #ld-pa vacants usually generate (2, 15) DFs
                            '2-16 shape',
                            'single',         #tables with a SINGLE VACANT generate different table structures
                            'otan'],
                  'posts': [np.nan]
                 }
    
    def __init__(self, df, filename, count):
        self.data = df
        self.file = filename
        self.index = count
        self.type = 'unknown'   #property for type of table
        self.vac_qty = 0        #property for number of vacants in the table
        self.doc_date = pd.to_datetime(filename.split('_')[1], format = '%Y%m%d')
        self.__get_type()         #call get_type() method at instantiation moment
        self.__get_vac_qty()      #call get_vac_qty() method at instantiation moment

        
#Method to get vacants TYPE
    def __get_type(self):      #method to determine the type of vacants table at hand
        if self.data.shape == (2, 14):
            self.type = 'LD-PA common'        #ld(30mar22)-pa(01abr22) vacants usually generate (2, 14) DFs
            assert self.type in self.table_types['vacants']
            
        elif self.data.shape == (2, 15):
            self.type = 'CM common'           #CM(31mar22) vacants usually generate (2, 15) DFs
            assert self.type in self.table_types['vacants']

        elif self.data.shape == (2, 16):
            self.type = '2-16 common'        #xxx vacants usually generate (2, 16) DFs
            assert self.type in self.table_types['vacants']

#         elif _____:
#             self.type = 'single'        #07MAR22. 16 columnas. Tienen "código OTAN" en el encabezado.
#             assert self.type in self.types 
            
#         elif _____:
#             self.type = 'otan'        #07MAR22. 16 columnas. Tienen "código OTAN" en el encabezado.
#             assert self.type in self.types            
            
        else:
            pass
            #03mar22. POSTS in international organizations
            #03mar22. VACANTS "Cuerpos Comunes"
            #03mar22. POSTS LD (Agustín)
            #03mar22. POSTS CM (Forzoso - Capote)
            #03mar22. VACANTS "Armada"
            #03mar22. VACANTS "EA"
            #17mar22. POSTS "OC"
        
    def __get_vac_qty(self):       #method to extract number of vacants in the table
        pass

    
#Block of methods to extract VACANTS
    def extract_vacs(self):        #method to extract vacants. Calls the appropriate private method
        if self.type == 'LD-PA common':
            return self.__extract_t1()

        elif self.type == t2:
            return self.__extract_t2()
            
        else: pass
    
    def __extract_t1(self):        #method to extract vacants in the type 1 table
        col_list =[]                                      #instanciamos la lista de columnas vacía que se poblará con los 'split' de cada columna del df de esta iteración
        string = [np.nan] * self.data.shape[1]
        col = [np.nan] * self.data.shape[1]
        
        if isinstance(self.data.iloc[1, 0], str):                    #n_vacs define el número de vacantes que se esperan (num de vacantes separadas por \r en la 1a col del DF)
            self.vac_qty = len(self.data.iloc[1, 0].split('\r'))
            
        elif isinstance(self.data.iloc[1, 0], float):
            self.vac_qty = 1
            
        else:
            print('DF-' + str(i) + 'found a ' + str(type(self.data.iloc[1, 0])) + ' in iloc [0, 1]')

        for j in range(self.data.shape[1]):                      #iteramos cada columna (str separado por '\r') del df para convertirlo en una lista de valores de la columna

            string[j] = self.data.iloc[1, j]

            if type(string[j]) == str:
                col[j] = string[j].split('\r')            #columna resultante de la separación de 'string'

                if len(col[j]) == self.vac_qty:
                    col_list.append(col[j])               #si la columna tiene exactamente el mismo número de registros que el número de vacantes del df, adjuntamos la columna

                elif math.ceil(len(col[j])/2) == self.vac_qty:
                    col_list.append(col[j][0::2])         #si la columna tiene el doble (redondeado) de registros que el número de vacantes del df, adjuntamos la columna cada dos espacios

                else:                                     #si no tiene el mismo número, lanzamos la lógica para colocar todos los valores posibles en ese registro y marcamos el error
                    unique = set(col[j])
                    values = list(unique)
                    values.append(np.nan)
                    col_list.append([values] * self.vac_qty)
            
            elif type(string[j]) == np.float64:
                col[j] = string[j]
                col_list.append(col[j])
                
            else:
                col_list.append([np.nan] * self.vac_qty)
    
        col_map = {'n_vac': col_list[0],
                   'uco': col_list[1],
                   'ciu': col_list[2],
                   'pt': col_list[3],
                   'asig': col_list[4],
                   'ta': col_list[5],
                   'empleo': col_list[6],
                   'efun': col_list[7],
                   'cursos': col_list[8],
                   't_max': col_list[9],
                   't_min': col_list[10],
                   'fecha_cob': col_list[11],
                   'csce': col_list[12],
                   'obs': col_list[13]
                  }
        
        data = pd.DataFrame(col_map)
        data['fecha_pub'] = self.doc_date
        
        return data
    
    
    def __extract_t2(self):        #method to extract vacants in the type x table
        pass
    
    def __extract_tx(self):        #method to extract vacants in the type x table
        pass
    
    
#Block of methods to extract POSTS
    def extract_posts(self):        #method to extract vacants. Calls the appropriate private method
        if self.type == t99:
            self.__extract_t99()

        elif self.type == t98:
            self.__extract_t98()
            
        else: pass
    
    def __extract_t99(self):        #method to extract vacants in the type 1 table
        pass
    
    def __extract_t98(self):        #method to extract vacants in the type x table
        pass
    
    def __extract_txx(self):        #method to extract vacants in the type x table
        pass
    

#Methods to analyze unprocessed tables

    def get_unprocessed(self):
        info = {
            'file': [self.file],
            'df_index': [self.index],
            'shape': [self.data.shape],
            'vac_qty': [self.vac_qty],
            'data': [self.data]
        }
        
        data = pd.DataFrame(info)
        
        return data

In [82]:
#este bloque busca los BOD en PDF en la carpeta de documentos (docs_dir), los compara con los documentos que ya se procesaron en su día guardados como .pkl (pkld_list) y genera una lista
#de nombres de archivos a procesar (proc_docs)

docs_dir = 'D:/jaume/Datasets/BOD/'
pkld_dir = 'D:/jaume/Jupyter Notebooks/Vacantes Scraper/ScrapedData/'

docs_list = []    #lista de documentos en la carpeta de entrada
pkld_list = []    #lista de documentos ya procesados y transformados a pkl
proc_docs = []    #lista de documentos aún por procesar

# iterate over files in that directory
for filename in os.scandir(docs_dir):
    if filename.is_file():
        docs_list.append(filename.name.split('_')[1])
        
# iterate over files in that directory
for filename in os.scandir(pkld_dir):
    if filename.is_file():
        pkld_list.append(filename.name.split('.')[0])

#seriales de documentos a procesar
pending_list = [doc for doc in docs_list if doc not in pkld_list]

# list of files to porcess
for filename in os.scandir(docs_dir):
    if filename.name.split('_')[1] in pending_list:
        proc_docs.append(filename)

In [83]:
unprocessed_tables = pd.DataFrame()
vacants = pd.DataFrame()
posts = pd.DataFrame()


for doc in proc_docs:
    
    print('Processing ' + doc.name + '...')
    file_pdf = docs_dir + doc.name
    read_pdf = tabula.read_pdf(file_pdf, pages = 'all', silent = True)

    for count, df in enumerate(read_pdf):
        table = VacsTable(df, doc.name, count)
        if table.type in table.table_types['vacants']:
            data = table.extract_vacs()
            vacants = pd.concat([vacants, data]).reset_index(drop = True)

        elif table.type in table.table_types['posts']:
            data = table.extract_posts()
            posts = pd.concat([posts, data]).reset_index(drop = True)

        else:
            assert table.type == 'unknown'
            
            data = table.get_unprocessed()         #method to analyze unprocessed dataframes           
            unprocessed_tables = pd.concat([unprocessed_tables, data]).reset_index(drop = True)     #adds DF with additional info to discarded_tables list of DFs.


# vacants = pd.concat(vacants_tables).reset_index(drop = True)
# posts = pd.concat(posts_tables).reset_index(drop = True)

# print(str(vacants.shape[0]) + ' vacants extracted.')
# print(str(posts.shape[0]) + ' posts extracted.')
# print(len(discarded_tables) + ' tables not processed.')

Processing BOD_20210324_57.pdf...
Processing BOD_20210326_59.pdf...


In [84]:
vacants

,n_vac,uco,ciu,pt,asig,ta,empleo,efun,cursos,t_max,t_min,fecha_cob,csce,obs,fecha_pub
0,02500,CUARTEL GENERAL DEL MANDO DE TROPAS DE MONTAÑA,50083231,5WA1E/001,LD,Z,SBMY,NaN,"[TM:E, PA:E, nan]",NaN,NaN,"[25-04-2021, 17-05-2021, nan]","412,70","244,604,656,959",2021-03-24
1,02501,"BCG DE LA BRIGADA «ALMOGÁVARES» VI, DE PARACAI...",50027882,5WA1E/001,LD,Z,SBMY,NaN,"[TM:E, PA:E, nan]",NaN,NaN,"[25-04-2021, 17-05-2021, nan]","412,70","244,959",2021-03-24
2,02502,BATALLON DE ZAPADORES VI,50056870,5WA1E/001,LD,Z,SBMY,NaN,"[TM:E, PA:E, nan]",NaN,NaN,"[25-04-2021, 17-05-2021, nan]","412,70","244,609,959",2021-03-24
3,02503,GRUPO DE ARTILLERIA DE CAMPAÑA VI,50091180,5WA1E/001,LD,Z,SBMY,NaN,"[TM:E, PA:E, nan]",NaN,NaN,"[25-04-2021, 17-05-2021, nan]","412,70","244,607,959",2021-03-24
4,02504,BATALLON DE CG. DE LA BRIGADA «GALICIA» VII,50027771,5WA1E/002,LD,Z,SBMY,NaN,"[TM:E, PA:E, nan]",NaN,NaN,"[25-04-2021, 17-05-2021, nan]","374,87","244,959",2021-03-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,05402,DIRECCION DE INFRAESTRUCTURA,54630000,5WB88/004,PA,C,CAP,CONST,NaN,"[8, nan]",NaN,NaN,"338,86","656,960",2021-03-26
512,05403,JEFATURA DEL MANDO DE APOYO LOGISTICO,57005389,5WB89/002,PA,C,CAP,TL/EL,NaN,"[8, nan]",NaN,NaN,"338,86","468,656,960",2021-03-26
513,05404,JEFATURA DEL MANDO DE APOYO LOGISTICO,57005389,5WB87/011,PA,C,CAP,ARMT.,NaN,"[8, nan]",NaN,NaN,"338,86","656,960",2021-03-26
514,05405,COMANDANCIA DE OBRAS N.o 3 ZARAGOZA,54630008,5WC6A/005,PA,C,CAP a TTE,CONST,NaN,"[8, nan]",NaN,NaN,"291,07",960,2021-03-26


# Below, code for reviewing possible skipped data during logic application:

In [50]:
unprocessed_tables

,file,df_index,shape,vac_qty,data
0,BOD_20210324_57.pdf,0,"(47, 3)",0,"Núm. 57 Miércoles, 24 de marzo ..."
1,BOD_20210324_57.pdf,1,"(0, 1)",0,Empty DataFrame Columns: [BOLETÍN OFICIAL DE...
2,BOD_20210324_57.pdf,2,"(0, 1)",0,Empty DataFrame Columns: [BOLETÍN OFICIAL DE...
3,BOD_20210324_57.pdf,3,"(0, 1)",0,Empty DataFrame Columns: [BOLETÍN OFICIAL DE...
4,BOD_20210324_57.pdf,4,"(0, 1)",0,Empty DataFrame Columns: [BOLETÍN OFICIAL DE...
...,...,...,...,...,...
225,BOD_20210326_59.pdf,128,"(0, 1)",0,Empty DataFrame Columns: [DOCUMENTACIÓN QUE SE...
226,BOD_20210326_59.pdf,129,"(2, 4)",0,Unna...
227,BOD_20210326_59.pdf,130,"(1, 7)",0,CAMP...
228,BOD_20210326_59.pdf,131,"(23, 24)",0,EFECTOSEMPLEO CUERPO ESCALA ESP. OBS.FUND. ...


In [85]:
#conditions which select the tables to review
min_cols = 10
at_least_min_cols = [t[1] >= min_cols for t in unprocessed_tables['shape']]
not_empty = [t.shape[0] != 0 for t in unprocessed_tables['data']]

tables_to_review = unprocessed_tables[not_empty and at_least_min_cols]
tables_to_review

,file,df_index,shape,vac_qty,data
10,BOD_20210324_57.pdf,10,"(23, 12)",0,Unnamed: 0 ...
21,BOD_20210324_57.pdf,24,"(1, 12)",0,"NUMERONVúACmANTE. CENTRO, UNIDAD, ORGANISMO7..."
22,BOD_20210324_57.pdf,25,"(5, 24)",0,"CSCENVúACmANTE. 57 TIEMPOSNUMERO CENTRO, UNI..."
53,BOD_20210324_57.pdf,62,"(8, 24)",0,CSCENúm. 57 OBS.VACANTE TIEMPOSNUMERO CENTRO...
54,BOD_20210324_57.pdf,63,"(2, 23)",0,"TIEMPOSNUMERO OBS.VACANTE CENTRO, UNIDAD, OR..."
55,BOD_20210324_57.pdf,65,"(8, 23)",0,"TIEMPOSNUMERO OBS.VACANTE CENTRO, UNIDAD, OR..."
56,BOD_20210324_57.pdf,66,"(6, 23)",0,"TIEMPOSNUMERO OBS.VACANTE CENTRO, UNIDAD, OR..."
57,BOD_20210324_57.pdf,67,"(2, 23)",0,"TIEMPOSNUMERO OBS.VACANTE CENTRO, UNIDAD, OR..."
58,BOD_20210324_57.pdf,68,"(12, 23)",0,"TIEMPOSNUMERO OBS.VACANTE CENTRO, UNIDAD, O..."
67,BOD_20210324_57.pdf,79,"(3, 12)",0,CARACTERNUM. UNIDAD/CENTRO/ORG...


## To get a CSV file:
(only use after getting _tables_to_review_)

In [93]:
#EXPORT to a CSV
tables_to_review.to_csv(date.today().strftime(format = '%Y%m%d') + '__to_review.csv')